# Sodium Conductor Project
### Chem 4PB3 - Winter 2024
##### Cameron Gurwell
<hr></hr>

<p>For most standard applications of structure optimisation
(e.g. <strong>CASTEP, VASP, Quantum Espresso</strong>), a high powered supercomputer is necessary
to crunch the large amounts of data and numbers. This project was aimed at investigating the recent
developments into low power structure optimisation using ML.</p>

<p><strong>MACE</strong>, the optimisation library used inside of <strong>Python</strong>, was developed
using the <strong>ASE</strong> framework and uses complex ANSATZ picking to construct wavefunctions.
<strong>MACE-MP-0</strong>, the model used ontop of MACE, was developed by the
<strong>Materials Project</strong> and <strong>MACE</strong> as a pretrained model to pick ANSATZ for
89 elements on the periodic table.</p>

<p>This project used the above materials to create the below figure using structure optimisation
and a library called <strong>BVLain</strong> to construct sodium ion pathways through a novel conductor
intially developed by the <strong>Mozharivskyj Group</strong> and investigated by the
<strong>Goward Group</strong>.</p><br>

> Novikov, S., et al., <em>Inorganic Chemistry</em> <strong>2023</strong>, <em>62</em>, 16068-16076.

<br>
<p><em><strong>Most calculations and work was completed using an AMD Ryzen 3700x
CPU and an AMD Radeon 6800 GPU.</em></strong>

<hr></hr>

### Na<sub>4</sub>Sn<sub>2</sub>Ge<sub>5</sub>O<sub>16</sub>

![Sodium Conductor with Channels](https://github.com/Camgur/4PB3_Sodium_Conductor/blob/89063e302fde4656a65c12b8e674236dbfd52be6/Figures/Na4Sn2Ge5O16.png?raw=true)

<p>Cloning the repo from github is this easy! Use <code>!ls</code> to list the directories directly in the <code>/content</code> section of this <strong>Colab</strong> session.</p>

In [ ]:
!git clone https://github.com/Camgur/4PB3_Sodium_Conductor.git
!ls

<p>Installing <strong>dependencies</strong>. The most current version of <strong>Torch</strong> is not compatible with the version of <strong>MACE</strong> we are using for this code.

* <strong>Spglib</strong> is used to determine spacegroups for the crystal structures we are using.
* <strong>NGLview</strong> is the viewer we are using to visualise CIF and Trajectory files
* <strong>BVLain</strong> is the BVEL library we are using to construct the sodium channels

<p>Note: <strong>FullProfSuite</strong> is the program used to calculate the BVEL in the paper. That said, <strong>BVLain</strong> is available in <strong>Python</strong>, so we will use it even though it may not be a one-to one comparison.

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install mace-torch
!pip install spglib
!pip install nglview==3.0.3
!pip install bvlain

In [3]:
import torch
import numpy as np
import mace

import matplotlib as mpl
import matplotlib.pyplot as plt

from mace.calculators import mace_mp, MACECalculator
from ase.calculators.loggingcalc import LoggingCalculator
from ase.optimize import BFGS

from ase.visualize import view
from ase import build, units, atoms
from ase.io import read, write, Trajectory
from ase.io.animation import write_gif

from ase.constraints import ExpCellFilter, StrainFilter, UnitCellFilter
from ase.spacegroup.symmetrize import FixSymmetry, check_symmetry
from spglib import get_spacegroup

<p>GPUs enable faster processing of the structure optimisation. If this reads <code>False</code>, please switch to GPU.<br>
</br>Unless you hate yourself.</p>

In [ ]:
# Printing True if GPU is connected
print(torch.cuda.is_available())

<p>This is the ASE framework list of standard structures. These are pre-formed structures of <code>atoms</code> which can be used out of the box.</p>

In [ ]:
# List of generic structures to be imported
from ase.collections import g2
print(g2.names)

## Water
<hr>
<p>The standard example of structure optimisation uses the water molecule. Here, the standard is imported from the above <code>g2</code> database.</p>
<p>It is important to note that the standard unit to store data about a molecule or crystal structure is the <code>atoms</code> object, which <strong>ASE</strong> is built around.

In [ ]:
# Building a water molecule
atoms = build.molecule('H2O')
view(atoms, viewer='x3d')

<p>A calculator may be attached to the <code>atoms</code> object using <code>atoms.calc</code>. Other similar options may be attached to an <code>atoms</code> object like this. Importantly, however, is that the calculator is not a global value.</p>
<br>
<p>Any model can be called from the <code>model=</code> argument, including a model that you have created. Additionally, the <strong>data type</strong> (either <em>float32</em> or <em>float64</em>) may be passed, and the <strong>device</strong> must be set (in this case, <em>cuda</em>). Note: if no <strong>data type</strong> is passed, the model used will automatically determine the best type to be used.</p>

In [ ]:
# Setting the calculator for the atom structure
calculator = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cuda')
atoms.calc = calculator

print(atoms.get_potential_energy())

## MACE-MP-0

<p>Here, the model we will use for the remainder of the simulation is downloaded. Interestingly, the difference in energy between both models is about 4 $\frac{kJ}{mol}$, or 0.04 eV. This is not a very significant difference, however. Plus, the validity of the <em>medium</em> model is unknown.</p>

In [ ]:
# Setting the MACE-MP-0 Calculator
calculator = MACECalculator(model_paths='/content/4PB3_Sodium_Conductor/2024-01-07-mace-128-L2_epoch-199.model',
                            dispersion=False, device='cuda', default_dtype='float64')
atoms.calc = calculator

print(atoms.get_potential_energy())

<p>Here, the data output is tracked for later plotting by appling a second <code>log_calc</code>. Additionally, the atoms values for each iteration are saved in a <em>Trajectory</em> file. <strong>BFGS</strong> (Broyden–Fletcher–Goldfarb–Shanno algorithm) is the standard for structure optimisation in QM.

In [ ]:
# Track Data
nsteps = []
energies = []
log_calc = LoggingCalculator(calculator)

print(atoms.symbols)

# Set Log
atoms.calc = log_calc

# Optimise
opt = BFGS(atoms, trajectory='/content/4PB3_Sodium_Conductor/Resources/Optimisation_Stuff/H2O.traj')

<p>Here, <code>opt.run</code> is invoked to start the structure optimisation using the above parameters attached to <code>opt</code>. The <em>fmax</em> parameter is the sum of forces on the atoms object each iteration in $\frac{eV}{Å}$. When the forces reach this value, the optimisation will terminate.</p>

In [ ]:
# Run Optimise
opt.run(fmax=1e-4)
print('Finished!!!')

In [ ]:
# Plot Out
plt.figure(figsize=(10,10))
log_calc.plot(markers=['r-', 'b-'], energy=True, lw=2)
plt.show()

In [ ]:
print(atoms.get_potential_energy())

<p>As a demonstration of the <strong>ASE framework</strong>, the vibrational modes of H<sub>2</sub>O are calculated using <strong>MACE-MP-0</strong>. These can be saved as trajectories using <code>write_mode</code>.

In [ ]:
from ase.vibrations import Vibrations

# Running analysis of the vibrational modes
# of H2O
vib = Vibrations(atoms)
vib.run()
vib.summary()
vib.write_mode(n=None, kT=0.02585199101165164, nimages=60)

<p>Here, the <em>/Content/</em> directory of the <strong>Colab</strong> sesion is being parsed for any file that ends with <em>.traj</em>. Then, the trajectory file is used to construct a <em>.mp4</em> file, which can be played.

In [ ]:
import os

# Set the directory to connect
directory = '/content/'

# Iterate over the listed files in the directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)

  if f.endswith('.traj'): # If it is a trajectory file, it will proceed
    traj = Trajectory(f)
    write_gif(f.strip('.traj') + '.mp4', traj, interval=33, rotation=('270x,90y,0z')) #Writing an mp4 file

In [ ]:
from IPython.display import Video
Video('/content/vib.6.mp4', embed=True) # Vibrational mode 6 for H2O

# If the above fails:
# Video('/content/4PB3_Sodium_Conductor/vib.6.mp4', embed=True)

<p><strong>Google Colab</strong> only allows external widgets to be used if the below is invoked. In my experience, it must be repeated shortly before any cell that uses <strong>NGLview</strong>, so a repeat cell is included before each <strong>NGLview</strong> cell.

In [18]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import nglview as nv

traj = Trajectory('/content/vib.6.traj')
view(traj, viewer='ngl')

##Importing Crystals

<p>Here, the crystal file <code>Na4Sn2Ge5O16_fixed.cif</code> is imported. Atoms outside of the unit cell were discarded to enable the smooth completion of the optimisation. Additionally, <code>atoms.set_constraint(FixSymmetry(atoms))</code> was used to maintain the symmetry operations of the spacegroup (pbcn).

In [20]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Importing Stuff
atoms = read('/content/4PB3_Sodium_Conductor/Resources/Na4Sn2Ge5O16_fixed.cif')

# Set Calculator
atoms.calc = calculator
print(atoms.symbols)

# Track Data
nsteps = []
energies = []
log_calc = LoggingCalculator(calculator)

# Set Log
atoms.calc = log_calc

# Set Cell filter (preserve unit cell ratioe or symmetry)
# atoms = ExpCellFilter(atoms, hydrostatic_strain=False)
atoms.set_constraint(FixSymmetry(atoms))

view(atoms, viewer='ngl')

<p>The <code>atoms.rattle</code> function is used to apply a gaussian deviation to the atomic coordinates within one Å.

In [ ]:
from ase.visualize.plot import plot_atoms

# Begin Plot
fig, axarr = plt.subplots(1, 2, figsize=(10, 7))

plot_atoms(atoms, axarr[0], radii=0.3, rotation=('0x,0y,0z')).set_title('Pristine')


atoms.rattle(1) #Angstom, gaussian


plot_atoms(atoms, axarr[1], radii=0.3, rotation=('0x,0y,0z')).set_title('Rattled')

plt.show()

In [ ]:
print("Cell size before: ", atoms.cell)

<p>This cell will take about one minute to run on a V100 GPU.</p>

In [ ]:
# Run Optimise
opt = BFGS(UnitCellFilter(atoms), trajectory='/content/trajectory.traj')
opt.run(fmax=1e-4)

print("Cell size after : ", atoms.cell)
print("Spacegroup: ", get_spacegroup((atoms.cell, atoms.get_scaled_positions(), atoms.numbers), symprec=1e-2))
atoms.write('/content/optimisation.cif')

# Plot Out
plt.figure(figsize=(10,10))
log_calc.plot(markers=['r-', 'b-'], energy=True, lw=2)
plt.show()

print('Finished!!!')

## Iteration

<p>In order to run iterations, the <code>atoms</code> object must be set each time, and the parameters applied to it as well. Importantly, the seed must be set to a new value, or the rng value applied must be changed. If this does not occur, the rattle will instead default to the same value for each iteration after the first, leading to a bad result akin to running the same optimisation over and over again. This is, however, extremely useful in running tandem optimisations.</p>

Example code for running iterations:

```
for i in range(iter):
    print('Iteration: ', i+1)
    # Set Savestate
    trajsave = '/content/Trajectories/Trajectory_1_'
    trajsave += str(i)
    trajsave += '.traj'
    cifsave = '/content/TCIF/Crystal_1_'
    cifsave += str(i)
    cifsave += '.cif'

    #set atoms
    atoms = read('/content/drive/MyDrive/Chem_4PB3/Resources/Na4Sn2Ge5O16_fixed.cif')
    atoms.set_constraint(FixSymmetry(atoms))
    atoms.rattle(stdev=1, seed=i) # seed required to generate different randomness
    atoms.calc = calculator

    # Optimise
    opt = BFGS(UnitCellFilter(atoms), trajectory=trajsave)
    opt.run(fmax=1e-4)
    atoms.write(cifsave)

    # Output Params
    print('\n\n')
    print("Cell size after : ", atoms.cell)
    print("Spacegroup: ", get_spacegroup((atoms.cell, atoms.get_scaled_positions(), atoms.numbers), symprec=1e-2))
    print('Iteration: ', i+1)
```


To create a compilation of trajectories:
```
# Import Trajectories
traj = []
for i in range(15):
    traject = '/content/Trajectories/Trajectory_1_' + str(i) + '.traj'
    traj.append(Trajectory(traject))

# Draft Compiled Trajectory
write('/content/Compiled.traj', '')
for i in range(len(traj[0])):
    atom = None
    atoms = []
    for n in range(len(traj)):
        atoms.append(traj[n][i])
    atom = atoms[0] + atoms[1] + atoms[2] + atoms[3] + atoms[4] + atoms[5] + atoms[6] + atoms[7] +\
            atoms[8] + atoms[9] + atoms[10] + atoms[11] + atoms[12] + atoms[13] + atoms[14]
    with Trajectory('/content/Compiled.traj', mode='a') as trajectory:
        trajectory.write(atom)
```
Annoyingly, it is surprisingly difficult to work with trajectory files containing more than one <code>atoms</code> object! <code>atoms</code> objects cannot be added together except by addition, but the '+=' function doesn't seem to work to loop together additions. Rather, it appears that each atom object must be added individually and manually.

In [25]:
from google.colab import output
output.enable_custom_widget_manager()

<p>It is recommended that you switch the color scheme to <em>atomindex</em> to easily differentiate between the individual <code>atoms</code> objects contained within this trajectory file. The slider under <em>ball size</em> may be used to switch between iterations.<p>
<p>This dataset had one outlier convergence (rattle = 1 A):</p>

In [ ]:
traj = Trajectory('/content/4PB3_Sodium_Conductor/Resources/Compiled_out.traj')
view(traj, viewer='ngl')
# Use atom index colour to show the overlap converging

# write('/content/drive/MyDrive/Chem_4PB3/Resources/Compiled.mp4', traj, interval=33, rotation=('45x,45y,35z'))

In [ ]:
length = []
for i in range(15):
  traj = Trajectory('/content/4PB3_Sodium_Conductor/Resources/Trajectories/Trajectory_1_' + str(i) + '.traj')
  length.append(len(traj))
print(length)
print('Trajectory 13: ', length[13], '\n\n\n')

traj = Trajectory('/content/4PB3_Sodium_Conductor/Resources/Trajectories/Trajectory_1_13.traj')
print('Initial Energy (eV): ', traj[0].get_total_energy())
print('Final Energy (eV): ', traj[-1].get_total_energy())

With the outlier removed:

In [28]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
traj = Trajectory('/content/4PB3_Sodium_Conductor/Resources/Compiled.traj')
view(traj, viewer='ngl')
# Use atom index colour to show the overlap converging

# write('/content/4PB3_Sodium_Conductor/Resources/Compiled.mp4', traj, interval=33, rotation=('45x,45y,35z'))

<p>Here, we are pulling the individual trajectories and assessing its energy value.</p>

In [ ]:
# Set energies
energies = []
for i in range(15):
    if i != 13: # Delete Outlier
        traj = Trajectory('/content/4PB3_Sodium_Conductor/Resources/Trajectories/Trajectory_1_' + str(i) + '.traj')
        energy = []
        for n in range(170):
            energy.append(traj[n].get_total_energy())
        energies.append(energy)
        print('Iteration: ', i + 1)

energies = np.array(energies)

# Average by iteration
average = []
stdev = []
for i in range(170):
    average = np.append(average, np.average(energies[:, i]))
    stdev = np.append(stdev, np.std(energies[:, i]))

In [ ]:
# Set linspace for dataset
x1 = np.linspace(1, 170, 170)
x2 = np.linspace(1, 20, 20)
y = average

# Plotting the set
fig, ax = plt.subplots(figsize=(10,10), layout="tight")
ax.plot(x1, average, color='black')
plt.fill_between(x1, y-stdev, y+stdev, color='red', alpha=.3) # This is the standard deviation
ax.set_xlabel('Iteration')
ax.set_ylabel('Average Energy (eV)')
ax.set_title('Average Energy Over Iteration Number', pad=30)

plt.show()

<p>Zoomed in on first 20 iterations:</p>

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), layout="tight")
ax.plot(x2, average[:20], color='black')
# ax.errorbar(np.linspace(1, 170, 170), average, yerr=stdev, fmt='none', color='red')
plt.fill_between(x2, y[:20]-stdev[:20], y[:20]+stdev[:20], color='red', alpha=.3)
ax.set_xlabel('Iteration')
ax.set_xticks(x2)
ax.set_ylabel('Average Energy (eV)')
ax.set_title('Average Energy Over Iteration Number', pad=30)

plt.show()

## Channels

<p>The channels were created on the home PC using <strong>BVLain</strong>. That said, the page for the library is incredibly short and frankly bad currently. The result is pretty good, but the validity compared to a dedicated program like <strong>FullProfSuite</strong> should be determined.

In [33]:
from bvlain import Lain

# Actually Using: BVlain
# https://pypi.org/project/bvlain/
# https://bvlain.readthedocs.io/en/latest/index.html

# Initialise File
file = '/content/4PB3_Sodium_Conductor/Resources/Optimisation_Stuff/Optimisation_0_3.cif'

# Set Calculator
calc = Lain(verbose=True)

# Set State
st = calc.read_file(file)

params = {'mobile_ion': 'Na1+',    # mobile specie
		  'r_cut': 10.0,           # cutoff for interaction between the mobile species and framework
		  'resolution': 0.1,	   # distance between the grid points
		  'k': 100                 # maximum number of neighbors to be collected for each point
}

<p>This crashes:

I don't know why. My home PC can handle this fine.</p>

In [ ]:
# Run Distributions
_ = calc.bvse_distribution(**params)
# _ = calc.void_distribution(**params)

# Perform Percolation Analysis
calc.percolation_barriers(encut = 5.0)

# Create Savestate
savestate = file.replace('.cif', '_bvel')

# Write Grid File
calc.write_grd(filename = savestate, task = 'bvse')  # saves .grd file
calc.write_cube(filename = savestate, task = 'bvse')  # saves .cube file

# Check for Mismatches
table = calc.mismatch(r_cut = 4.0)
# print(table.to_string())
print('Finished!!!')

# MAKE SURE TO SET ISOSURFACE TO 0.4
# MUST SET THE ISOSURFACE TO NEGATIVE

## Displaying the Ion Channels

<p>The <em>.grd</em> files were taken from <strong>BVLain</strong> and imported as an isosurface on the conductor CIF file in <strong>VESTA</strong> software. Isosurface was set to 0.4 and the phase was set to negative only to display the below figures.

In [ ]:
from IPython.display import Image
Image('/content/4PB3_Sodium_Conductor/Figures/Na4Sn2Ge5O16.png', embed=True, width='1000')

In [ ]:
Image('/content/4PB3_Sodium_Conductor/Figures/Na4Sn2Ge5O16_NoAtoms.png', embed=True, width='1000')

In [ ]:
Image('/content/4PB3_Sodium_Conductor/Figures/Na4Sn2Ge5O16_NoPolyhedra.png', embed=True, width='1000')

# Citations

In [ ]:
# MACE-MP-0
@misc{batatia2023foundation,
    title={A foundation model for atomistic materials chemistry},
    author={Ilyes Batatia and Philipp Benner and Yuan Chiang and Alin M. Elena and Dávid P. Kovács and Janosh Riebesell and Xavier R. Advincula and Mark Asta and William J. Baldwin and Noam Bernstein and Arghya Bhowmik and Samuel M. Blau and Vlad Cărare and James P. Darby and Sandip De and Flaviano Della Pia and Volker L. Deringer and Rokas Elijošius and Zakariya El-Machachi and Edvin Fako and Andrea C. Ferrari and Annalena Genreith-Schriever and Janine George and Rhys E. A. Goodall and Clare P. Grey and Shuang Han and Will Handley and Hendrik H. Heenen and Kersti Hermansson and Christian Holm and Jad Jaafar and Stephan Hofmann and Konstantin S. Jakob and Hyunwook Jung and Venkat Kapil and Aaron D. Kaplan and Nima Karimitari and Namu Kroupa and Jolla Kullgren and Matthew C. Kuner and Domantas Kuryla and Guoda Liepuoniute and Johannes T. Margraf and Ioan-Bogdan Magdău and Angelos Michaelides and J. Harry Moore and Aakash A. Naik and Samuel P. Niblett and Sam Walton Norwood and Niamh O''Neill and Christoph Ortner and Kristin A. Persson and Karsten Reuter and Andrew S. Rosen and Lars L. Schaaf and Christoph Schran and Eric Sivonxay and Tamás K. Stenczel and Viktor Svahn and Christopher Sutton and Cas van der Oord and Eszter Varga-Umbrich and Tejs Vegge and Martin Vondrák and Yangshuai Wang and William C. Witt and Fabian Zills and Gábor Csányi},
    year={2023},
    eprint={2401.00096},
    archivePrefix={arXiv},
    primaryClass={physics.chem-ph}
}

# PyTorch
@incollection{NEURIPS2019_9015,
title = {PyTorch: An Imperative Style, High-Performance Deep Learning Library},
author = {Paszke, Adam and Gross, Sam and Massa, Francisco and Lerer, Adam and Bradbury, James and Chanan, Gregory and Killeen, Trevor and Lin, Zeming and Gimelshein, Natalia and Antiga, Luca and Desmaison, Alban and Kopf, Andreas and Yang, Edward and DeVito, Zachary and Raison, Martin and Tejani, Alykhan and Chilamkurthy, Sasank and Steiner, Benoit and Fang, Lu and Bai, Junjie and Chintala, Soumith},
booktitle = {Advances in Neural Information Processing Systems 32},
pages = {8024--8035},
year = {2019},
publisher = {Curran Associates, Inc.},
url = {http://papers.neurips.cc/paper/9015-pytorch-an-imperative-style-high-performance-deep-learning-library.pdf}
}

# Matplotlib
@Article{Hunter:2007,
  Author    = {Hunter, J. D.},
  Title     = {Matplotlib: A 2D graphics environment},
  Journal   = {Computing in Science \& Engineering},
  Volume    = {9},
  Number    = {3},
  Pages     = {90--95},
  abstract  = {Matplotlib is a 2D graphics package used for Python for
  application development, interactive scripting, and publication-quality
  image generation across user interfaces and operating systems.},
  publisher = {IEEE COMPUTER SOC},
  doi       = {10.1109/MCSE.2007.55},
  year      = 2007
}

# spglib
@misc{spglibv1,
  Author = {Atsushi Togo and Isao Tanaka},
  Title = {$\texttt{Spglib}$: a software library for crystal symmetry search},
  Eprint = {arXiv:1808.01590},
  howpublished = {\url{https://github.com/spglib/spglib}},
  year = {2018}
}

# NGLviewer
@article{10.1093/bioinformatics/btx789,
    author = {Nguyen, Hai and Case, David A and Rose, Alexander S},
    title = "{NGLview–interactive molecular graphics for Jupyter notebooks}",
    journal = {Bioinformatics},
    volume = {34},
    number = {7},
    pages = {1241-1242},
    year = {2017},
    month = {12},
    abstract = "{NGLview is a Jupyter/IPython widget to interactively view molecular structures as well as trajectories from molecular dynamics simulations. Fast and scalable molecular graphics are provided through the NGL Viewer. The widget supports showing data from the file-system, online data bases and from objects of many popular analysis libraries including mdanalysis, mdtraj, pytraj, rdkit and more.The source code is freely available under the MIT license at https://github.com/arose/nglview. Python packages are available from PyPI and bioconda. NGLview uses Python on the server-side and JavaScript on the client. The integration with Jupyter is done through the ipywidgets package. The NGL Viewer is embedded client-side to provide WebGL accelerated molecular graphics.}",
    issn = {1367-4803},
    doi = {10.1093/bioinformatics/btx789},
    url = {https://doi.org/10.1093/bioinformatics/btx789},
    eprint = {https://academic.oup.com/bioinformatics/article-pdf/34/7/1241/48914829/bioinformatics\_34\_7\_1241.pdf},
}

# BVLain (No official citation)
  author = Artem Dembitskiy
  Year = 2022
  URL = 'https://bvlain.readthedocs.io/en/latest/index.html'